In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/C247Project/project/

/content/drive/.shortcut-targets-by-id/1lO_lCKfnzu9fB3MDutIRFUOktcz9B9el/C247Project/project


In [ ]:
!pip install tensorflow

In [ ]:
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Permute, Bidirectional, BatchNormalization, Conv2D, LSTM, Dense, Dropout, ELU, Flatten, MaxPool2D, TimeDistributed, Dense
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import lecun_uniform
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

## Import Datasets

In [ ]:
X_test = np.load("Dataset/X_test.npy")
y_test = np.load("Dataset/y_test.npy")
X_train_valid = np.load("Dataset/X_train_valid.npy")
y_train_valid = np.load("Dataset/y_train_valid.npy")
person_train_valid = np.load("Dataset/person_train_valid.npy")
person_test = np.load("Dataset/person_test.npy")
X_synthetic = np.load("synthetic_data.npy")
y_synthetic = y_train_valid

## Adjusting the labels 
y_train_valid -= 769
y_test -= 769

# Constant
num_classes = 4

print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))
print ('Synthetic data shape: {}'.format(X_synthetic.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)
Synthetic data shape: (2115, 22, 1000)


## Function Definitions

In [ ]:
'''
    Preprocess data: 
    - Trim
    - Maxpool
    - Average + noise
    - Subsampling + noise
'''
def data_prep(X,y,sub_sample,average,noise):
    total_X = None
    total_y = None
    
    # Trimming the data (sample,22,1000) -> (sample,22,500)
    X = X[:,:,0:500]

    # Maxpooling the data (sample,22,1000) -> (sample,22,500/sub_sample)
    X_max = np.max(X.reshape(X.shape[0], X.shape[1], -1, sub_sample), axis=3)
    
    total_X, total_y = X_max, y
    
    # Averaging + noise 
    X_average = np.mean(X.reshape(X.shape[0], X.shape[1], -1, average),axis=3)
    X_average = X_average + np.random.normal(0.0, 0.5, X_average.shape)
    
    total_X = np.vstack((total_X, X_average))
    total_y = np.hstack((total_y, y))
    
    # Subsampling
    for i in range(sub_sample):
        X_subsample = X[:, :, i::sub_sample] + \
                            (np.random.normal(0.0, 0.5, X[:, :,i::sub_sample].shape) if noise else 0.0)
        total_X = np.vstack((total_X, X_subsample))
        total_y = np.hstack((total_y, y))
    
    return total_X,total_y


'''
    Random split training data into training and validation set
    Reshape training, validation, and test sets
'''
def rand_split_reshape_data(X_train_valid_prep, y_train_valid_prep, X_test_prep, y_test_prep, valid_size=1500):
    # First generating the training and validation indices using random splitting
    ind_valid = np.random.choice(len(X_train_valid_prep), valid_size, replace=False)
    ind_train = np.array(list(set(range(len(X_train_valid_prep))).difference(set(ind_valid))))

    # Creating the training and validation sets using the generated indices
    (x_train, x_valid) = X_train_valid_prep[ind_train], X_train_valid_prep[ind_valid]
    (y_train, y_valid) = y_train_valid_prep[ind_train], y_train_valid_prep[ind_valid]

    # Converting the labels to categorical variables for multiclass classification
    y_train = to_categorical(y_train, 4)
    y_valid = to_categorical(y_valid, 4)
    y_test = to_categorical(y_test_prep, 4)

    # Adding width of the segment to be 1
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
    x_valid = x_valid.reshape(x_valid.shape[0], x_valid.shape[1], x_train.shape[2], 1)
    x_test = X_test_prep.reshape(X_test_prep.shape[0], X_test_prep.shape[1], X_test_prep.shape[2], 1)

    # Reshaping the training and validation dataset
    x_train = np.swapaxes(x_train, 1,3)
    x_train = np.swapaxes(x_train, 1,2)
    x_valid = np.swapaxes(x_valid, 1,3)
    x_valid = np.swapaxes(x_valid, 1,2)
    x_test = np.swapaxes(x_test, 1,3)
    x_test = np.swapaxes(x_test, 1,2)

    return x_train, y_train, x_valid, y_valid, x_test, y_test
    

'''
    Helper function
    Random split training data into training and validation set
'''
def rand_split_data(X_train_valid_prep, y_train_valid_prep, X_test_prep, y_test_prep, valid_size=1500):
    # First generating the training and validation indices using random splitting
    ind_valid = np.random.choice(len(X_train_valid_prep), valid_size, replace=False)
    ind_train = np.array(list(set(range(len(X_train_valid_prep))).difference(set(ind_valid))))

    # Creating the training and validation sets using the generated indices
    (x_train, x_valid) = X_train_valid_prep[ind_train], X_train_valid_prep[ind_valid]
    (y_train, y_valid) = y_train_valid_prep[ind_train], y_train_valid_prep[ind_valid]

    return x_train, y_train, x_valid, y_valid, X_test_prep, y_test_prep


'''
    Helper function
    Convert labels (y) into categorical
'''
def convert_categorical(x_train, y_train, x_valid, y_valid, X_test_prep, y_test):
    # Converting the labels to categorical variables for multiclass classification
    y_train = to_categorical(y_train, 4)
    y_valid = to_categorical(y_valid, 4)
    y_test = to_categorical(y_test, 4)
    return x_train, y_train, x_valid, y_valid, X_test_prep, y_test


'''
    Helper function
    Reshape training, validation, and test data
'''
def reshape_data(x_train, y_train, x_valid, y_valid, x_test, y_test): 
    # Adding width of the segment to be 1
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
    x_valid = x_valid.reshape(x_valid.shape[0], x_valid.shape[1], x_train.shape[2], 1)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

    # Reshaping the training and validation dataset
    x_train = np.swapaxes(x_train, 1,3)
    x_train = np.swapaxes(x_train, 1,2)
    x_valid = np.swapaxes(x_valid, 1,3)
    x_valid = np.swapaxes(x_valid, 1,2)
    x_test = np.swapaxes(x_test, 1,3)
    x_test = np.swapaxes(x_test, 1,2)

    return x_train, y_train, x_valid, y_valid, x_test, y_test


'''
    Concatenate synthetic data into the training dataset
'''
def add_synthetic_data(x_train_valid, y_train_valid):
    # Stack original and synthetic data
    x_train_valid = np.vstack((x_train_valid, X_synthetic))
    y_train_valid = np.hstack((y_train_valid, y_synthetic))
    
    return x_train_valid, y_train_valid


'''
    Selects entries corresponding to selected person
'''
def sub_selection(idx, X_train, y_train, X_test, y_test, person_train_valid, person_test):
    train_idx = np.where(person_train_valid == idx)[0]
    test_idx = np.where(person_test == idx)[0]
    X_train = X_train[train_idx]
    y_train = y_train[train_idx]
    X_test = X_test[test_idx]
    y_test = y_test[test_idx]
    return X_train, y_train, X_test, y_test


'''
    Train model by (1) preprocess data, (2) split into training and validation sets
'''
def train(model, params, data, idx=None):
    epoch_num, batch_size = params
    X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test = data

    # Subject split
    if idx is not None:
        X_train_valid, y_train_valid, X_test, y_test = sub_selection(idx, X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test)

    # Preprocess Data: Training and testing data
    X_train_valid_prep, y_train_valid_prep = data_prep(X_train_valid,y_train_valid,2,2,True)
    X_test_prep, y_test_prep = data_prep(X_test,y_test,2,2,True)

    # Randomly split and reshape: Training Data 
    x_train, y_train, x_valid, y_valid, x_test, y_test = rand_split_reshape_data(X_train_valid_prep, y_train_valid_prep, X_test_prep, y_test_prep)

    model.fit(x_train, y_train, epochs=epoch_num, batch_size=batch_size, 
              validation_data=(x_valid, y_valid), shuffle=True, verbose=1)
    train_score = model.evaluate(x_train, y_train)
    test_score = model.evaluate(x_test, y_test)

    print('train {:s}: {:.3f}%'.format(model.metrics_names[1], train_score[1]*100))
    print('test {:s}: {:.3f}%'.format(model.metrics_names[1], test_score[1]*100))
    
    return train_score, test_score


'''
    Train model by (1) split into training and validation sets, (2) preprocess data
'''
def train_split_first(model, params, data, idx=None):
    epoch_num, batch_size = params
    X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test = data

    # Subject split
    if idx is not None:
        X_train_valid, y_train_valid, X_test, y_test = sub_selection(idx, X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test)
    
    train_valid_size = len(X_train_valid)
    valid_size = int(train_valid_size * 0.17)

    # Randomly split: Training Data 
    x_train, y_train, x_valid, y_valid, x_test, y_test = rand_split_data(X_train_valid, y_train_valid, X_test, y_test, valid_size)

    # Preprocess: Training, validation, and testing data
    x_train, y_train = data_prep(x_train,y_train,2,2,True)
    x_valid, y_valid = data_prep(x_valid, y_valid,2,2,True)
    x_test, y_test = data_prep(x_test, y_test,2,2,True)

    # Convert labels into categorical
    x_train, y_train, x_valid, y_valid, x_test, y_test = convert_categorical(x_train, y_train, x_valid, y_valid, x_test, y_test)

    # Reshape: Training, validation, and testing data
    x_train, y_train, x_valid, y_valid, x_test, y_test = reshape_data(x_train, y_train, x_valid, y_valid, x_test, y_test)

    model.fit(x_train, y_train, epochs=epoch_num, batch_size=batch_size, 
              validation_data=(x_valid, y_valid), shuffle=True, verbose=1)
    train_score = model.evaluate(x_train, y_train)
    test_score = model.evaluate(x_test, y_test)

    print('train {:s}: {:.3f}%'.format(model.metrics_names[1], train_score[1]*100))
    print('test {:s}: {:.3f}%'.format(model.metrics_names[1], test_score[1]*100))
    
    return train_score, test_score

'''
    Train model by (1) Add synthetic data, (2) split into training and validation sets, (3) preprocess data
'''
def train_split_first_with_synthetic(model, params, data, idx=None):
    epoch_num, batch_size = params
    X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test = data

    # Add synthetic data
    X_train_valid, y_train_valid = add_synthetic_data(X_train_valid, y_train_valid)

    # Subject split
    if idx is not None:
        X_train_valid, y_train_valid, X_test, y_test = sub_selection(idx, X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test)

    train_valid_size = len(X_train_valid)
    valid_size = int(train_valid_size * 0.18)

    # Randomly split: Training Data 
    x_train, y_train, x_valid, y_valid, x_test, y_test = rand_split_data(X_train_valid, y_train_valid, X_test, y_test, valid_size)

    # Preprocess: Training, validation, and testing data
    x_train, y_train = data_prep(x_train,y_train,2,2,True)
    x_valid, y_valid = data_prep(x_valid, y_valid,2,2,True)
    x_test, y_test = data_prep(x_test, y_test,2,2,True)

    # Cnvert labels into categorical
    x_train, y_train, x_valid, y_valid, x_test, y_test = convert_categorical(x_train, y_train, x_valid, y_valid, x_test, y_test)

    # Reshape: Training, validation, and testing data
    x_train, y_train, x_valid, y_valid, x_test, y_test = reshape_data(x_train, y_train, x_valid, y_valid, x_test, y_test)

    model.fit(x_train, y_train, epochs=epoch_num, batch_size=batch_size, 
              validation_data=(x_valid, y_valid), shuffle=True, verbose=1)
    train_score = model.evaluate(x_train, y_train)
    test_score = model.evaluate(x_test, y_test)

    print('train {:s}: {:.3f}%'.format(model.metrics_names[1], train_score[1]*100))
    print('test {:s}: {:.3f}%'.format(model.metrics_names[1], test_score[1]*100))
    
    return train_score, test_score 

## Model Setup

In [ ]:
def CNN_model(kernel_initializer, params):

    lr, dropout_rate = params

    channels_in_1, channels_out_1, kernel_size_1, stride_1 = (250,1,22), 25, (10, 1), 1
    pool_size_1, pool_stride_1 = (3, 1), 1
    channels_out_2, kernel_size_2, stride_2 = 50, (10, 1), 1
    pool_size_2, pool_stride_2 = (3, 1), 1
    channels_out_3, kernel_size_3, stride_3 = 100, (10, 1), 1
    pool_size_3, pool_stride_3 = (3, 1), 1
    channels_out_4, kernel_size_4, stride_4 = 200, (10, 1), 1
    pool_size_4, pool_stride_4 = (3, 1), 1

    model = Sequential(
        [
        Conv2D(filters=channels_out_1, kernel_size=kernel_size_1, padding='same', activation='elu', input_shape=channels_in_1),
        MaxPool2D(pool_size=pool_size_1, padding='same'),
        BatchNormalization(),
        Dropout(dropout_rate),

        Conv2D(filters=channels_out_2, kernel_size=kernel_size_2, padding='same', activation='elu'),
        MaxPool2D(pool_size=pool_size_2, padding='same'),
        BatchNormalization(),
        Dropout(dropout_rate),

        Conv2D(filters=channels_out_3, kernel_size=kernel_size_3, padding='same', activation='elu'),
        MaxPool2D(pool_size=pool_size_3, padding='same'),
        BatchNormalization(),
        Dropout(dropout_rate),

        Conv2D(filters=channels_out_4, kernel_size=kernel_size_4, padding='same', activation='elu'),
        MaxPool2D(pool_size=pool_size_4, padding='same'),
        BatchNormalization(),
        Dropout(dropout_rate),
         
        Flatten(),
        Dense(num_classes, activation='softmax'),
        ]
    )
    model.compile(loss='categorical_crossentropy', 
                  optimizer=Adam(learning_rate=lr),
                  metrics=[categorical_accuracy])
    return model

## Hyper parameter tuning

#### (1) Preprocess (2) Split

In [ ]:
epoch_num = 40
lrs = [1e-3, 8e-4, 6e-4, 4e-4]
bzs = [30, 64, 80]
drs = [0.4, 0.5, 0.6]

tuning_results = []

for lr in lrs:
    for bz in bzs:
        for dr in drs:
            curr_model = CNN_model(lecun_uniform(seed=42), [lr, dr])
            train_acc, test_acc = train(curr_model,
                                        [epoch_num, bz],
                                        [X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test]
                                        )
            tuning_results.append((lr, bz, dr, train_acc, test_acc))

Epoch 1/40
232/232 [==============================] - 3s 10ms/step - loss: 1.7237 - categorical_accuracy: 0.3556 - val_loss: 1.2308 - val_categorical_accuracy: 0.4393
Epoch 2/40
232/232 [==============================] - 2s 9ms/step - loss: 1.2791 - categorical_accuracy: 0.4682 - val_loss: 1.0379 - val_categorical_accuracy: 0.5733
Epoch 3/40
232/232 [==============================] - 2s 9ms/step - loss: 1.0924 - categorical_accuracy: 0.5547 - val_loss: 0.9225 - val_categorical_accuracy: 0.6293
Epoch 4/40
232/232 [==============================] - 2s 9ms/step - loss: 0.9801 - categorical_accuracy: 0.5994 - val_loss: 0.8699 - val_categorical_accuracy: 0.6247
Epoch 5/40
232/232 [==============================] - 2s 10ms/step - loss: 0.9125 - categorical_accuracy: 0.6356 - val_loss: 0.8494 - val_categorical_accuracy: 0.6393
Epoch 6/40
232/232 [==============================] - 2s 9ms/step - loss: 0.8183 - categorical_accuracy: 0.6766 - val_loss: 0.6635 - val_categorical_accuracy: 0.7487
Ep

In [ ]:
# CNN Model, train
# epoch_num = 40
# lrs = [1e-3, 8e-4, 6e-4, 4e-4]
# bzs = [30, 64, 80]
# drs = [0.4, 0.5, 0.6]

for res in tuning_results:
    print("lr={", res[0], "}", end=" ")
    print("bz={", res[1], "}", end=" ")
    print("dr={", res[2], "}", end=" ")
    print('train: {:.3f}%'.format(res[3][1]*100), end=" ")
    print('test: {:.3f}%'.format(res[4][1]*100))

# BEST TEST ACCURACY
# lr={ 0.001 } bz={ 30 } dr={ 0.6 } train: 90.948% test: 72.686%
# Sample training and validation loss at epoch 40
# loss: 0.2260 - categorical_accuracy: 0.9147 - val_loss: 0.0488 - val_categorical_accuracy: 0.9880

lr={ 0.001 } bz={ 30 } dr={ 0.4 } train: 99.928% test: 70.316%
lr={ 0.001 } bz={ 30 } dr={ 0.5 } train: 99.052% test: 70.485%
lr={ 0.001 } bz={ 30 } dr={ 0.6 } train: 90.948% test: 72.686%
lr={ 0.001 } bz={ 64 } dr={ 0.4 } train: 99.928% test: 69.131%
lr={ 0.001 } bz={ 64 } dr={ 0.5 } train: 98.477% test: 71.501%
lr={ 0.001 } bz={ 64 } dr={ 0.6 } train: 84.684% test: 67.946%
lr={ 0.001 } bz={ 80 } dr={ 0.4 } train: 99.871% test: 68.059%
lr={ 0.001 } bz={ 80 } dr={ 0.5 } train: 98.046% test: 70.993%
lr={ 0.001 } bz={ 80 } dr={ 0.6 } train: 88.161% test: 69.074%
lr={ 0.0008 } bz={ 30 } dr={ 0.4 } train: 99.971% test: 69.131%
lr={ 0.0008 } bz={ 30 } dr={ 0.5 } train: 98.247% test: 68.454%
lr={ 0.0008 } bz={ 30 } dr={ 0.6 } train: 86.882% test: 68.849%
lr={ 0.0008 } bz={ 64 } dr={ 0.4 } train: 99.986% test: 67.833%
lr={ 0.0008 } bz={ 64 } dr={ 0.5 } train: 96.379% test: 71.050%
lr={ 0.0008 } bz={ 64 } dr={ 0.6 } train: 87.069% test: 70.880%
lr={ 0.0008 } bz={ 80 } dr={ 0.4 } train: 99.957%

#### (1) Split (2) Preprocess

In [ ]:
epoch_num = 40
lrs = [1e-3, 8e-4, 6e-4, 4e-4]
bzs = [30, 64, 80]
drs = [0.4, 0.5, 0.6]

tuning_results = []

for lr in lrs:
    for bz in bzs:
        for dr in drs:
            curr_model = CNN_model(lecun_uniform(seed=42), [lr, dr])
            train_acc, test_acc = train_split_first(curr_model,
                                        [epoch_num, bz],
                                        [X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test]
                                        )
            tuning_results.append((lr, bz, dr, train_acc, test_acc))

Epoch 1/40
235/235 [==============================] - 4s 13ms/step - loss: 1.7626 - categorical_accuracy: 0.3595 - val_loss: 1.2976 - val_categorical_accuracy: 0.4338
Epoch 2/40
235/235 [==============================] - 2s 10ms/step - loss: 1.2672 - categorical_accuracy: 0.4697 - val_loss: 1.2435 - val_categorical_accuracy: 0.4652
Epoch 3/40
235/235 [==============================] - 3s 11ms/step - loss: 1.0948 - categorical_accuracy: 0.5537 - val_loss: 0.9859 - val_categorical_accuracy: 0.6128
Epoch 4/40
235/235 [==============================] - 3s 12ms/step - loss: 0.9692 - categorical_accuracy: 0.6115 - val_loss: 0.9443 - val_categorical_accuracy: 0.6086
Epoch 5/40
235/235 [==============================] - 3s 11ms/step - loss: 0.8563 - categorical_accuracy: 0.6596 - val_loss: 0.9671 - val_categorical_accuracy: 0.6114
Epoch 6/40
235/235 [==============================] - 3s 11ms/step - loss: 0.7805 - categorical_accuracy: 0.6913 - val_loss: 0.9452 - val_categorical_accuracy: 0.628

In [ ]:
# CNN Model, train_split_first
# epoch_num = 40
# lrs = [1e-3, 8e-4, 6e-4, 4e-4]
# bzs = [30, 64, 80]
# drs = [0.4, 0.5, 0.6]

for res in tuning_results:
    print("lr={", res[0], "}", end=" ")
    print("bz={", res[1], "}", end=" ")
    print("dr={", res[2], "}", end=" ")
    print('train: {:.3f}%'.format(res[3][1]*100), end=" ")
    print('test: {:.3f}%'.format(res[4][1]*100))

# BEST TEST ACCURACY
# lr={ 0.0006 } bz={ 64 } dr={ 0.5 } train: 99.473% test: 73.081%
# Sample training and validation loss at epoch 40
# loss: 0.5986 - categorical_accuracy: 0.7608 - val_loss: 0.7792 - val_categorical_accuracy: 0.6831

lr={ 0.001 } bz={ 30 } dr={ 0.4 } train: 99.986% test: 65.745%
lr={ 0.001 } bz={ 30 } dr={ 0.5 } train: 99.815% test: 70.880%
lr={ 0.001 } bz={ 30 } dr={ 0.6 } train: 93.223% test: 68.284%
lr={ 0.001 } bz={ 64 } dr={ 0.4 } train: 100.000% test: 65.068%
lr={ 0.001 } bz={ 64 } dr={ 0.5 } train: 99.075% test: 69.244%
lr={ 0.001 } bz={ 64 } dr={ 0.6 } train: 89.621% test: 68.284%
lr={ 0.001 } bz={ 80 } dr={ 0.4 } train: 100.000% test: 67.777%
lr={ 0.001 } bz={ 80 } dr={ 0.5 } train: 99.302% test: 68.115%
lr={ 0.001 } bz={ 80 } dr={ 0.6 } train: 89.251% test: 68.736%
lr={ 0.0008 } bz={ 30 } dr={ 0.4 } train: 100.000% test: 65.463%
lr={ 0.0008 } bz={ 30 } dr={ 0.5 } train: 99.473% test: 69.808%
lr={ 0.0008 } bz={ 30 } dr={ 0.6 } train: 92.298% test: 69.695%
lr={ 0.0008 } bz={ 64 } dr={ 0.4 } train: 100.000% test: 68.059%
lr={ 0.0008 } bz={ 64 } dr={ 0.5 } train: 99.644% test: 71.106%
lr={ 0.0008 } bz={ 64 } dr={ 0.6 } train: 89.408% test: 69.187%
lr={ 0.0008 } bz={ 80 } dr={ 0.4 } train: 99.

#### (1) Add synthetic data (2) Split (3) Preprocess

In [ ]:
epoch_num = 40
lrs = [1e-3, 6e-4, 2e-4]
bzs = [30, 64, 80]
drs = [0.4, 0.5, 0.6]

tuning_results = []

for lr in lrs:
    for bz in bzs:
        for dr in drs:
            curr_model = CNN_model(lecun_uniform(seed=42), [lr, dr])
            train_acc, test_acc = train_split_first_with_synthetic(curr_model,
                                        [epoch_num, bz],
                                        [X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test]
                                        )
            tuning_results.append((lr, bz, dr, train_acc, test_acc))

Epoch 1/40
463/463 [==============================] - 37s 77ms/step - loss: 1.6580 - categorical_accuracy: 0.3064 - val_loss: 1.3416 - val_categorical_accuracy: 0.3748
Epoch 2/40
463/463 [==============================] - 33s 72ms/step - loss: 1.3583 - categorical_accuracy: 0.3836 - val_loss: 1.2401 - val_categorical_accuracy: 0.3926
Epoch 3/40
463/463 [==============================] - 35s 76ms/step - loss: 1.2516 - categorical_accuracy: 0.4228 - val_loss: 1.2232 - val_categorical_accuracy: 0.4136
Epoch 4/40
463/463 [==============================] - 32s 68ms/step - loss: 1.2131 - categorical_accuracy: 0.4429 - val_loss: 1.2256 - val_categorical_accuracy: 0.4271
Epoch 5/40
463/463 [==============================] - 31s 67ms/step - loss: 1.1605 - categorical_accuracy: 0.4592 - val_loss: 1.1528 - val_categorical_accuracy: 0.4550
Epoch 6/40
463/463 [==============================] - 30s 66ms/step - loss: 1.1224 - categorical_accuracy: 0.4777 - val_loss: 1.1479 - val_categorical_accuracy:

In [ ]:
# CNN Model, train_split_first_with_synthetic
# epoch_num = 40
# lrs = [1e-3, 6e-4, 2e-4]
# bzs = [30, 64, 80]
# drs = [0.4, 0.5, 0.6]

for res in tuning_results:
    print("lr={", res[0], "}", end=" ")
    print("bz={", res[1], "}", end=" ")
    print("dr={", res[2], "}", end=" ")
    print('train: {:.3f}%'.format(res[3][1]*100), end=" ")
    print('test: {:.3f}%'.format(res[4][1]*100))

# BEST TEST ACCURACY
# lr={ 0.001 } bz={ 30 } dr={ 0.6 } train: 60.738% test: 71.670%
# Sample training and validation loss at epoch 40
# loss: 0.7815 - categorical_accuracy: 0.6179 - val_loss: 1.2580 - val_categorical_accuracy: 0.4609

lr={ 0.001 } bz={ 30 } dr={ 0.4 } train: 65.761% test: 67.269%
lr={ 0.001 } bz={ 30 } dr={ 0.5 } train: 64.709% test: 67.777%
lr={ 0.001 } bz={ 30 } dr={ 0.6 } train: 60.738% test: 71.670%
lr={ 0.001 } bz={ 64 } dr={ 0.4 } train: 65.494% test: 68.397%
lr={ 0.001 } bz={ 64 } dr={ 0.5 } train: 64.529% test: 67.720%
lr={ 0.001 } bz={ 64 } dr={ 0.6 } train: 59.707% test: 68.905%
lr={ 0.001 } bz={ 80 } dr={ 0.4 } train: 65.588% test: 70.372%
lr={ 0.001 } bz={ 80 } dr={ 0.5 } train: 64.103% test: 68.228%
lr={ 0.001 } bz={ 80 } dr={ 0.6 } train: 58.122% test: 67.777%
lr={ 0.0006 } bz={ 30 } dr={ 0.4 } train: 66.431% test: 68.115%
lr={ 0.0006 } bz={ 30 } dr={ 0.5 } train: 63.686% test: 68.115%
lr={ 0.0006 } bz={ 30 } dr={ 0.6 } train: 58.799% test: 69.187%
lr={ 0.0006 } bz={ 64 } dr={ 0.4 } train: 65.984% test: 66.591%
lr={ 0.0006 } bz={ 64 } dr={ 0.5 } train: 63.693% test: 67.494%
lr={ 0.0006 } bz={ 64 } dr={ 0.6 } train: 57.257% test: 66.648%
lr={ 0.0006 } bz={ 80 } dr={ 0.4 } train: 63.109%